In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta, datetime
from faker import Faker
f = Faker()

# from Supermarket_Code import Supermarket
# from Customer_Code import Customer


In [7]:
df = pd.read_csv('./data/weekly_loc1_data.csv', sep=",", index_col='timestamp', parse_dates=True)
df.sort_index(inplace = True)

# creating one minute intervals
df = df.groupby('cust_id').resample('1min').ffill()

# shifting to get location 2
df['location_2'] = df['location_1'].shift(-1)

#removing exit to exit state
df = df[df.location_1 != 'exit']
df = df[df.location_2 != 'exit']

In [8]:
# using pandas to create probability matrix dataframe
prob_df = pd.crosstab(df['location_1'],df['location_2'], normalize='index',)
prob_df.round(2)

#converting 'matrix' to dictionary of values only
TRANSITION_MATRIX = prob_df.round(2).to_dict(orient='index')

for key in TRANSITION_MATRIX.keys():
    TRANSITION_MATRIX[key] = list(TRANSITION_MATRIX[key].values())

# declare possible states in a list 

ALL_STATES = ['checkout', 'dairy', 'drinks', 'fruit', 'spices']

In [9]:
class Customer():
    """  Customer class from Damoon github - creates customer who makes decisions on where to go next 
    """
    def __init__(self, name, state):
        self.name = name
        self.state = state

    def __repr__(self):
        return f'Customer {self.name} in {self.state}'

    def next_state(self):
        '''
        Propagates the customer to the next state.
        Returns nothing.
        '''
        #self.state = random.choices(['checkout', 'dairy', 'drinks', 'fruit', 'spices'], weights= prob_df.loc[self.state])
        if self.is_active() == True:
            self.state = random.choices(population=ALL_STATES, weights=TRANSITION_MATRIX[self.state])[0]

    def is_active(self):
        if self.state == 'checkout':
            return False
        else:
            return True

In [10]:
class Supermarket:
    """
    an MCMC simulation that takes customers from Customer class and moves them throughout store during opening times
    """
    def __init__(self, market_name, opening,  closing):
        self.market_name = market_name
        self.opening = datetime.strptime(opening, '%H:%M')
        self.closing = datetime.strptime(closing, '%H:%M')
        self.active_customers = []
        self.customer_id = 0
        self.current_time = self.opening
        self.customer_list = []
        self.customer_df = pd.DataFrame()
        
    def open_store(self):
        """ 
        initialises the store to start counter and begin accepting customers  
        """
        self.open_for_business()
        self.timer()
        self.get_customers()

    def open_for_business(self) -> bool:
        """
        limiting setting to stop new customers entering store too late
        """
        while self.current_time <= self.closing - timedelta(minutes=3):
            return True
            
    def timer(self):
        """ 
        counter to run customer activity for each minute of opening time 
        """
        while self.current_time != self.closing: 
            self.current_time += timedelta(minutes=1)
            self.get_customers()
            self.move_customers()
            self.remove_customers()
            self.add_to_df()
        
    def get_customers(self):
        """ 
        allows random number of ne wcustomers to enter the store
        """
        if self.open_for_business() == True:
            n = np.random.poisson(5)
            for i in range(n):
                self.customer_id += 1
                self.customer_name = f.name()
                new_customer = Customer(self.customer_name, "entrance")
                self.customer_list.append(new_customer)
                self.active_customers.append(new_customer)

    def move_customers(self):
        """ 
        moves all customers active in the store to their next state
        """
        for customer in self.active_customers:
            customer.next_state()

    def remove_customers(self):
        """ 
        removes all inactive customers from the active list
        """
        for customer in self.active_customers:
            if customer.is_active == False:
                self.active_customers.remove(customer)
                
    def get_time(self):
        return datetime.strftime(self.current_time, '%H:%M')

    def add_to_df(self):
        """ 
        creates dataframe with time/customer/location for each minute 'freezeframe' for active customers
        """
        for customer in self.active_customers:
                self.customer_df = self.customer_df.append({'timestamp': self.get_time(),
                                                        'customer_name': str(customer.name),
                                                        'location': customer.state
                                                        }, ignore_index=True)

In [11]:
class Simulation():
    """ 
    class to take user input for store data and initiate the simulation
    """
    def start_simulation(self):
        """ 
        method to ask user for store data
        """
        self.store = input(f'What is the name of your store?')
        self.open = input(f'What time does your store open?')
        self.close = input(f'What time does your store close?')
        self.confirmation = input('Thank you. The simulation will run and output saved as CSV file. Do you wish to proceed [Y/N]?')

        if self.confirmation == 'Y':
            my_store = Supermarket(self.store, self.open, self.close)

            my_store.open_store()

            my_store.customer_df.set_index('timestamp', inplace=True)
            my_store.customer_df.to_csv('simulated_market_output.csv')

        else:
            print('Goodbye')

In [12]:
attempt1 = Simulation()

In [13]:
attempt1.start_simulation()